In [1]:
import torch
import torch.nn as nn
import PIL.Image
import ivy
import numpy as np
import jax


jax.config.update("jax_enable_x64", True)


class SimpleConvNet(nn.Module):
    """
    Simple Convolutional Neural Network
    """

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(26 * 26 * 10, 50),
            nn.ReLU(),
            nn.Linear(50, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
        )
        
        self.apply(self.initialize_weights)

    def forward(self, x):
        return self.layers(x)

    def predict(self, inp):
        """predict digit for input"""
        self.eval()
        with torch.no_grad():
            raw_output = self(inp)
            _, pred = torch.max(raw_output, 1)
            return pred
        
    def initialize_weights(self, module):
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)

In [2]:
model = SimpleConvNet()

In [3]:
img = PIL.Image.open("0.png")
np_img = np.array(img)
tensor_img = torch.from_numpy(np_img).float()
tensor_img = tensor_img.unsqueeze(0).unsqueeze(0)
tensor_img = torch.nn.functional.interpolate(tensor_img, size=28, mode='bicubic', align_corners=False)

In [4]:
jax_graph = ivy.transpile(model, to="haiku", args=(tensor_img,))

2023-07-24 16:11:39.617508: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/rishab/anaconda3/envs/ivy/lib/python3.10/site-packages/ivy/func_wrapper.py:242: UserWarning: Creating many views will lead to overhead when performing inplace updates with this backend
  warnings.warn(


In [5]:
jax_graph = ivy.transpile(model, to="flax", args=(tensor_img,))

/home/rishab/anaconda3/envs/ivy/lib/python3.10/site-packages/ivy/func_wrapper.py:242: UserWarning: Creating many views will lead to overhead when performing inplace updates with this backend
  warnings.warn(


TypeError: to_flax_module.<locals>.TranspiledFlaxModule.__init__() got an unexpected keyword argument 'lazy'